# Movies

This notebook was originally authored by Abhijit Dasgupta and was adapted from [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) by Wes McKinney

## Objectives

* What are the highest rated movies?
* What is the best movie for date night?
* Which movies do men and women disagree on the most?

In [ ]:
import pandas as pd
import os
engine='python'

### Reading in the data using `merge`

In [ ]:
unames = ['user_id','gender','age','occupation','zip']
users = pd.read_table(os.path.join('data','movies','users.dat'), 
                      sep='::', header=None, names=unames,engine=engine)
   
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(os.path.join('data','movies','ratings.dat'), 
                        sep='::', header=None, names=rnames,engine=engine)

mnames = ['movie_id', 'title','genres']
movies = pd.read_table(os.path.join('data','movies','movies.dat'), 
                       sep='::', header=None, names=mnames,engine=engine)
data = pd.merge(pd.merge(ratings, users), movies)


In [ ]:
data.head()

## What is the highest rated movie?

In [ ]:
tmp = data[['title','rating']]
tmp.head()

In [ ]:
mean_rating = tmp.groupby('title').mean()
print(mean_rating.describe())
mean_rating.head(10)

In [ ]:
# view the top ten sorted by rating
mean_rating.sort_values(by='rating', ascending=False).head(10)

Seems a bit odd?  What's wrong with this picture?

In [ ]:
# view review counts
mean_rating = tmp.groupby('title')['rating'].agg(['mean','count']) 
mean_rating.sort_values(by='mean', ascending=False).head(10)

### Filter our Movies

Only look at movies that have had at least 1000 ratings.

In [ ]:
mask = mean_rating['count'] > 1000
mean_rating[mask].head()

In [ ]:
mean_rating[mask].sort_values(by='mean', ascending=False).head(10)

## What is the best movie for both men and women?

We would like to creat another data frame of our data that contains mean ratings with movie totals as row lables and gender as colunm lables.

In [ ]:
mean_ratings = pd.pivot_table(data, 'rating', index='title', columns ='gender', aggfunc='mean')
mean_ratings.head(10)

In [ ]:
mask.head(10)

But this has **all** the movies, not just the ones with the largest **count**.

Notice:

- The DataFrame `mean_ratings` has the `title` as the index.
- The `mask` also has `title` as the index.

In [ ]:
top_mean_ratings = mean_ratings.ix[mask]
top_mean_ratings.head()

What are the top rated movies by women?

In [ ]:
top_female = top_mean_ratings.sort_values('F', ascending=False)
top_female.head()

What are the top rated by men?

In [ ]:
top_male = top_mean_ratings.sort_values('M', ascending=False)
top_male.head(5)

### Which ones do men and women differ on the least, i.e., date night?

In [ ]:
top_mean_ratings = top_mean_ratings.assign(diff = abs(top_mean_ratings.F - top_mean_ratings.M))
top_mean_ratings.sort_values(by='diff', ascending=True).head(10)

### What's the worst movie for date night?

In [ ]:
top_mean_ratings.sort_values('diff', ascending=False).head(10)